In [ ]:
# !pip install ddict
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import threading
import time
import datetime
import json
import numpy as np
from ddict import DotAccessDict
import paho.mqtt.client as mqtt
import time

In [ ]:
n = 5

In [ ]:


device_config = 'config_device_'+ str(n)+'.txt'
device_id = 'config_device_'+str(n)

with open(device_config) as json_file:
    conf = DotAccessDict(json.load(json_file))
    conf_device = conf[device_id][0]
i = 0 
ls =np.array([])


def printit(): 
#     t = threading.Timer((60/conf_device.freq_per_minute), printit)
#     t.start()
    ts = time.time()
    st = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
    global i 
    global ls
    
    if len(ls)>= conf_device.freq_per_minute:
        ls = np.array([])
    
    
    if np.count_nonzero(ls>=conf_device.thershold) >= conf_device.extreme_count:
        x = np.random.randint(low=conf_device.minima, high=conf_device.max_1)
        ls = np.append(ls,x)
    else:
        x = np.random.randint(low=conf_device.minima, high=conf_device.max_2)
        ls = np.append(ls,x)
        
    i += 1   
    measure_value = x
    output = {'device_id':conf_device.id,
            'measure_value':measure_value,
            'time_stamp':st,
            'term':i,
            'count_threshold':np.count_nonzero(ls>=conf_device.thershold),
             'ls':ls.tolist()
             }

    return(json.dumps(output))
#     if i >=conf_device.loops:
#         t.cancel()






#define call back function
def on_log(client, userdata, level, buf):
    print('log: '+buf)
    
def on_connect(client, userdata, flags, rc):
    if rc==0:
        print("Connected ok")
    else:
        print("Bad connection Returned code= ",rc)

def on_disconnect(client, userdata, flags, rc =0):
        print("Disconnected result code "+str(rc))
        
def on_publish_succ(client, userdata, flags, rc,msg ):
    if msg.rc == mqtt.MQTT_ERR_SUCCESS :
        print("good")

def on_message(client,userdata,msg):
    topic = msg.topic
    m_decode = str(msg.payload.decode("utf-8"))
    print("message recieved",m_decode)
        
# broker = "test.mosquitto.org"
broker = "127.0.0.1"
client = mqtt.Client("test1")   #create new instatnce

           
client.on_connect = on_connect   #bind to callback function
client.on_disconnect = on_disconnect
client.on_publish_succ = on_publish_succ
client.on_log = on_log
client.on_message = on_message

print("Connecting to broker",broker)
client.connect(broker,1883,60)
client.subscribe("device_1")
client.loop_start()  #start loop

while True:
    output = printit()
    msg = client.publish("temperature",output)
    if msg.rc == mqtt.MQTT_ERR_SUCCESS :
        print("Published Successfully")
        time.sleep(60//conf_device.freq_per_minute)


client.loop_stop() #stop loop
client.disconnect()